In [1]:
import json
import pandas as pd
import os

# Define ubicación actual
#src_path = "./src"
#os.chdir(src_path)

# Cargar datos del archivo JSON
json_path = "../data/scraped_data.json"
with open(json_path, "r", encoding="utf-8") as f:
    datos = json.load(f)

fecha = datos["fecha"]
fx_data = datos.get("fx", {})
crude_data = datos.get("crudo", {})
rates_data = datos.get("tasas", {})

# Usar directamente los símbolos ISO como nombres de columna para FX
fx_row = {"Fecha": fecha}
for symbol, value in fx_data.items():
    if symbol == "DX-Y.NYB":
        fx_row["DXY"] = value
    elif symbol.endswith("=X"):
        fx_row[symbol.replace("=X", "")] = value
    else:
        fx_row[symbol] = value

# Para tasas, renombramos a US10Y
rates_row = {"Fecha": fecha}
if "USY" in rates_data:
    rates_row["USY"] = rates_data["USY"]

# Crudo permanece igual
crude_row = {"Fecha": fecha, **crude_data}

# Función robusta para insertar fila solo si la fecha no existe
def append_row_by_symbol(csv_path, row_dict):
    new_row = pd.DataFrame([row_dict])
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)

        # Evitar duplicados por fecha
        if "Fecha" in df.columns and row_dict["Fecha"] in df["Fecha"].astype(str).values:
            print(f"⚠️ La fecha {row_dict['Fecha']} ya existe en {os.path.basename(csv_path)}. No se agregó.")
            return df.tail(3)

        # Alinear columnas a las del archivo
        aligned_row = {col: new_row.iloc[0].get(col, None) for col in df.columns}
        aligned_row = pd.DataFrame([aligned_row])
        df = pd.concat([df, aligned_row], ignore_index=True)
        df = df.loc[:, ~df.columns.duplicated()]
    else:
        df = new_row

    df.to_csv(csv_path, index=False)
    return df.tail(3)

# Aplicar a los CSV
fx_tail = append_row_by_symbol("../data/fx_data.csv", fx_row)
crude_tail = append_row_by_symbol("../data/crude_prices.csv", crude_row)
rates_tail = append_row_by_symbol("../data/rates_data.csv", rates_row)

# Mostrar resumen
print("✔️ Últimos registros añadidos:")
display(fx_tail)
display(crude_tail)
display(rates_tail)


✔️ Últimos registros añadidos:


,Fecha,BRL,CLP,COP,KRW,ZAR,INR,MXN,PEN,PLN,...,RUB,THB,TRY,HUF,ILS,DXY,ARS,CNY,IDR,MYR
866,16/05/25,5.6677,943.630005,4170.500000,1398.099976,18.0312,85.552002,19.451,3.6862,3.82,...,80.875000,33.23,38.852699,360.5,3.5504,100.976997,1142.0,7.2085,16434.0,4.292
867,17/05/25,5.6625,943.630005,4180.470215,1398.099976,18.0312,85.552002,19.451,3.6862,3.82,...,80.875000,33.23,38.852699,360.5,3.5504,101.089996,1142.0,7.2085,16419.0,4.292
868,18/05/25,5.6625,943.630005,4180.470215,1398.099976,18.0303,85.552002,19.451,3.6862,3.82,...,80.675003,33.23,38.852699,360.5,3.5504,100.976997,1142.0,7.2085,16434.0,4.292


,Fecha,WTI,Brent
865,16/05/25,62.490002,65.389999
866,17/05/25,62.490002,65.410004
867,18/05/25,62.490002,65.389999


,Fecha,USY
866,16/05/25,4.441
867,17/05/25,4.441
868,18/05/25,4.441
